Define $\kappa(\sigma_1, \sigma_2)$ as $\text{P}[Y \ge -X | X \ge 0]$ where $X$ is normal mean $0$ variance $\sigma_1^2$ and $Y$ is indpendent normal mean $0$ variance $\sigma_2^2$.

In [1]:
import numpy.random
import numpy as np
import pandas as pd
import sympy
from sympy import cos, exp, factorial, gamma, loggamma, pi, sin, sqrt

In [2]:
rng = numpy.random.default_rng(seed=12345678903141592653589793)


In [3]:
def l1l2auc_empirical(n: int, *, nreps :int = 1000000) -> float:
    """
    Empirical estimate of L1L2 AUC.

    :param n: dimension we are working in
    :param nreps: number of repetitions to simulate, > 0
    :return: empirical estimate of kappa(s1, s2)
    """
    n = int(n)
    assert n > 0
    nreps = int(nreps)
    assert nreps > 0
    r_mean = float(sqrt(n - 1/2))
    r_std = float(sqrt(1/2))
    s_mean = float(sqrt(2 * n / pi))
    s_std = float(sqrt(1 - 3 / pi))
    r1 = rng.normal(size=nreps) * r_std + r_mean
    r2 = rng.normal(size=nreps) * r_std + r_mean
    s1 = rng.normal(size=nreps) * s_std + s_mean
    s2 = rng.normal(size=nreps) * s_std + s_mean
    return np.mean((r1 >= r2) == ((r1 * s1) >= (r2 * s2)))


In [4]:
l1l2auc_empirical(n=20)

0.886922

In [5]:
def kappa_empirical(s1: float, s2: float, *, nreps :int = 1000000) -> float:
    """
    Empirical estimate of
    $\kappa(\sigma_1, \sigma_2)$ as $\text{P}[Y \ge -X | X \ge 0]$ 
    where $X$ is normal mean $0$ variance $\sigma_1^2$ and $Y$ is 
    indpendent normal mean $0$ variance $\sigma_2^2$.

    :param s1: variance 1, > 0
    :param s2: variance 2, > 0
    :param nreps: number of repetitions to simulate, > 0
    :return: empirical estimate of kappa(s1, s2)
    """
    s1 = float(s1)
    s2 = float(s2)
    assert s1 > 0
    assert s2 > 0
    nreps = int(nreps)
    assert nreps > 0
    x = -np.abs(rng.normal(size=nreps) * np.sqrt(s1))
    y = rng.normal(size=nreps) * np.sqrt(s2)
    return np.mean(y >= x)


In [6]:
# kappa(1, 1) should limit out to 0.75
k_1_1_est = kappa_empirical(1, 1)
assert np.abs(k_1_1_est - 0.75) < 1e-3

k_1_1_est

0.750929

In [7]:
# kappa(a, b) should equal kappa(s a, s b) for s > 0
k_10_10_est = kappa_empirical(10, 10)

assert np.abs(k_10_10_est - 0.75) < 1e-3

k_10_10_est

0.749926

In [8]:
# kappa(1, b) should go to 1/2 as b gets large
k_1_large_est = kappa_empirical(1, 1e+5)
assert np.abs(k_1_large_est - 0.5) < 1e-2

k_1_large_est

0.501132

In [9]:
# kappa(1, 1/b) should go to 1 as 1/b gets large
k_1_small_est = kappa_empirical(1, 1e-5)
assert np.abs(k_1_small_est - 1) < 1e-2

k_1_small_est

0.998977

In [10]:
def kappa_heuristic(s1, s2):
    """
    Heuristic estimate of
    $\kappa(\sigma_1, \sigma_2)$ as $\text{P}[Y \ge -X | X \ge 0]$ 
    where $X$ is normal mean $0$ variance $\sigma_1^2$ and $Y$ is 
    indpendent normal mean $0$ variance $\sigma_2^2$.

    :param s1: variance 1, > 0
    :param s2: variance 2, > 0
    :return: heuristic estimate of kappa(s1, s2)
    """
    assert s1 > 0
    assert s2 > 0
    h_exp = 2/3
    return (1 + 1/(1 + (s2/s1)**h_exp))/2


In [11]:
for b in (0.01, 0.1, 1, 10, 100):
    h_est = kappa_heuristic(1, b)
    e_est = kappa_empirical(1, b)
    print(f"empirical est kappa(1, {b}) ~ {e_est}")
    print(f"heuristic est kappa(1, {b}) ~ {h_est}")

empirical est kappa(1, 0.01) ~ 0.968255
heuristic est kappa(1, 0.01) ~ 0.9778214910278492
empirical est kappa(1, 0.1) ~ 0.902751
heuristic est kappa(1, 0.1) ~ 0.9113724848182869
empirical est kappa(1, 1) ~ 0.7509
heuristic est kappa(1, 1) ~ 0.75
empirical est kappa(1, 10) ~ 0.598495
heuristic est kappa(1, 10) ~ 0.5886275151817132
empirical est kappa(1, 100) ~ 0.532626
heuristic est kappa(1, 100) ~ 0.5221785089721508


$\kappa(2 ((1 - 3 / \pi) / n), 2 \sqrt{2 / \pi} (1/(2 n)))$

In [12]:
s1 = 2 * (1 - 3 / pi)
s2 = 2 * sqrt(2 / pi) * (1/2)

In [13]:
s1

2 - 6/pi

In [14]:
s2

1.0*sqrt(2)/sqrt(pi)

In [15]:
kappa_empirical(float(s1), float(s2))

0.602957